In [1]:
# Initialise Ray
import ray

In [2]:
ray.init()

2020-11-28 01:03:14,143	INFO services.py:1090 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '172.17.0.3',
 'raylet_ip_address': '172.17.0.3',
 'redis_address': '172.17.0.3:38188',
 'object_store_address': '/tmp/ray/session_2020-11-28_01-03-13_596501_918/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-11-28_01-03-13_596501_918/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2020-11-28_01-03-13_596501_918',
 'metrics_export_port': 59587,
 'node_id': '9b8176facf87b08c5ee6adad21f50119116b4663'}

# Ray Tune Quickstart

In [4]:
from ray import tune

In [5]:
def objective(step, alpha, beta):
    return (0.1 + alpha * step / 100)**(-1) + beta * 0.1


def training_function(config):
    # Hyperparameters
    alpha, beta = config["alpha"], config["beta"]
    for step in range(10):
        # Iterative training function - can be any arbitrary training procedure.
        intermediate_score = objective(step, alpha, beta)
        # Feed the score back back to Tune.
        tune.report(mean_loss=intermediate_score)


analysis = tune.run(
    training_function,
    config={
        "alpha": tune.grid_search([0.001, 0.01, 0.1]),
        "beta": tune.choice([1, 2, 3])
    })

print("Best config: ", analysis.get_best_config(
    metric="mean_loss", mode="min"))

# Get a dataframe for analyzing trial results.
df = analysis.results_df

2020-11-28 01:06:37,301	WARNING function_runner.py:539 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2020-11-28 01:06:37,331	WARNING tune.py:401 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.
2020-11-28 01:06:37,338	INFO logger.py:200 -- pip install 'ray[tune]' to see TensorBoard files.
2020-11-28 01:06:37,338	WARNING logger.py:342 -- Could not instantiate TBXLogger: No module named 'tensorboardX'.
2020-11-28 01:06:37,339	ERROR syncer.py:63 -- Log sync requires rsync to be installed.


Trial name,status,loc,alpha,beta
training_function_f7049_00000,RUNNING,,0.001,2


Result for training_function_f7049_00000:
  date: 2020-11-28_01-06-37
  done: false
  experiment_id: 99abf19449f249ab95072cad24256b8b
  experiment_tag: 0_alpha=0.001,beta=2
  hostname: 0a1efe644149
  iterations_since_restore: 1
  mean_loss: 10.2
  neg_mean_loss: -10.2
  node_ip: 172.17.0.3
  pid: 1033
  time_since_restore: 0.0001468658447265625
  time_this_iter_s: 0.0001468658447265625
  time_total_s: 0.0001468658447265625
  timestamp: 1606525597
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: f7049_00000
  
Result for training_function_f7049_00001:
  date: 2020-11-28_01-06-37
  done: false
  experiment_id: bcd8a1b7414246d4bab5bf1df4bb4ba4
  experiment_tag: 1_alpha=0.01,beta=2
  hostname: 0a1efe644149
  iterations_since_restore: 1
  mean_loss: 10.2
  neg_mean_loss: -10.2
  node_ip: 172.17.0.3
  pid: 1029
  time_since_restore: 0.00022149085998535156
  time_this_iter_s: 0.00022149085998535156
  time_total_s: 0.00022149085998535156
  timestamp: 1606525597
  timesteps_sinc

Trial name,status,loc,alpha,beta,loss,iter,total time (s),neg_mean_loss
training_function_f7049_00000,TERMINATED,,0.001,2,10.191,10,0.0339675,-10.191
training_function_f7049_00001,TERMINATED,,0.01,2,10.1108,10,0.0493445,-10.1108
training_function_f7049_00002,TERMINATED,,0.1,1,9.27431,10,0.0347655,-9.27431


2020-11-28 01:06:37,965	INFO tune.py:439 -- Total run time: 0.66 seconds (0.62 seconds for the tuning loop).


Best config:  {'alpha': 0.1, 'beta': 1}


# Shutdown

In [3]:
ray.shutdown()